In [1]:
import os 
os.chdir(
    '../'
)

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/Praveenmurali986/dl_cnn_classifier.mlflow \
MLFLOW_TRACKING_USERNAME=Praveenmurali986 \
MLFLOW_TRACKING_PASSWORD=d89a7d050ba93761ce9cf62bb95d4d961bfbb5ba \

In [ ]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/Praveenmurali986/dl_cnn_classifier.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="Praveenmurali986"
os.environ['MLFLOW_TRACKING_PASSWORD']="d89a7d050ba93761ce9cf62bb95d4d961bfbb5ba"

In [2]:
os.getcwd()

'c:\\Users\\PRAVEEN\\Desktop\\dl_cnn_classifier'

In [3]:
import tensorflow as tf

model=tf.keras.models.load_model('artifacts/training/model.h5')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    params_image_size:list
    all_params:dict
    mlflow_uri:str
    params_batch_size:int 


In [5]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories,save_json

class ConfigurationManager():
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
    
    def get_validation_config(self):
        eval_config=EvaluationConfig(
            path_of_model=Path('artifacts/training/model.h5'),
            training_data=Path('artifacts/data_ingestion/PetImages'),
            params_image_size=self.params.IMAGE_SIZE,
            mlflow_uri='https://dagshub.com/Praveenmurali986/dl_cnn_classifier.mlflow',
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [6]:
from deepClassifier.entity import TrainingConfig
import tensorflow as tf
from pathlib import Path
from deepClassifier.utils.common import save_json
import mlflow.keras
import mlflow
from urllib import urlparse

class Evaluation:
    def __init__(self,config) -> None:
        self.config=config
    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path:Path):
        return tf.keras.models.load_model(path)
    def evaluate(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
    def save_score(self):
        scores={'loss':self.score[0],'accuracy': self.score[1]}
        save_json(path=Path('scores.json'),data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'loss':self.score[0],'accuracy':self.score[1]})
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")




In [ ]:
config=ConfigurationManager()
eval_config=config.get_validation_config()
evaluation=Evaluation(eval_config)
evaluation.evaluate()
evaluation.save_score()
evaluation.log_into_mlflow()